In [1]:
from openai import OpenAI

# 어시스턴트 생성로직 한번만 실행해야 함. 실행후에 주석처리 요망

client = OpenAI()

assistant = client.beta.assistants.create(
name="Book Assistant",
instructions="You help users with their question on the files they upload.",
model="gpt-4o-mini"",
tools=[{"type":"file_search"}],
)
assistant

# assistant_id = "asst_13eo2MdAQaNGc4XTBhTHlOW3" # 기업 주식을 사도 되는지 물어보는 어시스턴트
#assistant_id ="asst_0HebldM6GtgdVEQPszxsTi8O" # 책 내용분석 어시스턴트



In [2]:
client = OpenAI()

thread = client.beta.threads.create(
    messages = [
        {
            "role":"user",
            "content": "I want you to help me with this file",
        }
    ]
)
thread

Thread(id='thread_kvF28sKo7CjUmji79b3Ekbz6', created_at=1727925773, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None), _request_id='req_23c975cd30cecffa448076a7bb8b45a1', __exclude_fields__={'__exclude_fields__', '_request_id'})

In [3]:
file = client.files.create(
    file=open("./files/chapter_one.pdf", "rb"),
    purpose="assistants",
)
file

FileObject(id='file-deZwR72sgtr6FuJkGKka9wp7', bytes=69355, created_at=1727925777, filename='chapter_one.pdf', object='file', purpose='assistants', status='processed', status_details=None, _request_id='req_7e0228fe2ef50d63ae2a66075c86c85b', __exclude_fields__={'__exclude_fields__', '_request_id'})

In [4]:
client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Please refer to the uploaded file for my question.",
    attachments=[
        {
            "file_id":file.id,
            "tools":[
                {
                    "type":"file_search",
                },
            ],
        },
    ],
)

Message(id='msg_ANxL0AIW5W6oms8k95MxH1es', assistant_id=None, attachments=[Attachment(file_id='file-deZwR72sgtr6FuJkGKka9wp7', tools=[AttachmentToolAssistantToolsFileSearchTypeOnly(type='file_search')])], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Please refer to the uploaded file for my question.'), type='text')], created_at=1727925781, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_kvF28sKo7CjUmji79b3Ekbz6', _request_id='req_d6cb74190a4d4b894dd11361712d660b', __exclude_fields__={'__exclude_fields__', '_request_id'})

In [13]:

run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)
run

Run(id='run_QH4VmSqmJpga87D0g1QX16PX', assistant_id='asst_0HebldM6GtgdVEQPszxsTi8O', cancelled_at=None, completed_at=None, created_at=1727925877, expires_at=1727926477, failed_at=None, incomplete_details=None, instructions='You help users with their question on the files they upload.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4o', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_kvF28sKo7CjUmji79b3Ekbz6', tool_choice='auto', tools=[FileSearchTool(type='file_search', file_search=FileSearch(max_num_results=None, ranking_options=FileSearchRankingOptions(score_threshold=0.0, ranker='default_2024_08_21')))], truncation_strategy=TruncationStrategy(type='auto', last_messages=None), usage=None, temperature=1.0, top_p=1.0, tool_resources={}, _request_id='req_47dfe13bc29126a6d9f276efa1cdd45b', __exclude_fields__={'__exclude_fields__', '_request_id'})

In [5]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id
    )

def send_messages(thread_id, content):
    return client.beta.threads.messages.create(
        thread_id=thread_id,
        role="user",
        content=content
    )

def get_messages(thread_id):
    messages = client.beta.threads.messages.list(
        thread_id=thread_id
    )
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")


In [14]:
get_run(run.id, thread.id).status

'failed'

In [12]:
get_messages(thread.id)

user: I want you to help me with this file
user: Please refer to the uploaded file for my question.
user: I want to know where Winston lives and how his apartment looks like.


In [11]:
send_messages(thread.id, "I want to know where Winston lives and how his apartment looks like.")

Message(id='msg_H0PCLXcWG36YgUbcfNquCoMj', assistant_id=None, attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='I want to know where Winston lives and how his apartment looks like.'), type='text')], created_at=1727925844, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='user', run_id=None, status=None, thread_id='thread_kvF28sKo7CjUmji79b3Ekbz6', _request_id='req_09726e4bebee6acea3b65d2e40c4311a', __exclude_fields__={'__exclude_fields__', '_request_id'})